# Problem

We need to repeatedly choose between N possible actions so as to maximize the total reward over a sequence of M choices. Each action results in a random reward drawn from an unknown distribution specific to that action. In this notebook, these are normal distributions with randomly assigned means and variances (but that can be easily changed).

# Parameter setup

In [1]:
actions = 10 # number of actions N (with normally distributed rewards)
steps = 5000 # number of choices made in 1 simulation, M
simulations = 1000 # number of times the simulation is repeated

# Simulation setup

In [2]:
# Import libraries
import random
import numpy as np

In [3]:
# Function to select action using UCB1 strategy
# 4 parameters: actions - number of actions, step - the step in the simulation,
# mean_reward - list of current approximations of mean rewards for all actions
# times_tried - list of current numbers that each action had been tried
def select_action(actions, step, mean_reward, times_tried):
    
    if step < actions: # try every action once first
        action = step
    else:
        scores = [] # score actions based on their reward and number of times tried
        for action in range(actions):
            scores.append(mean_reward[action] + np.sqrt(2 * np.log(step) / times_tried[action]))
        action = scores.index(max(scores))

    return action

In [4]:
# Function to run simulation once
def run_simulation(actions, steps):

    # Initialize parameters
    mean = [random.random() for i in range(actions)] # means rewards for each of the possible actions
    stdev = [random.random() for i in range(actions)] # standard deviations of rewards for each possible action
    actual_best = mean.index(max(mean)) # action with the highest mean reward (for assessing algorithm performance)

    best = random.randrange(actions) # randomly guess "best" action
    # (alternatively, one could try all possible actions once and start with the one that gives highest reward)
    
#    print ("\n Initial best action guess: {:4d}\n".format(best))
    times_tried = [0 for i in range(actions)] # times each action has been tried
    mean_reward = [0. for i in range(actions)] # mean reward approximation for each attempted action
    total_reward = 0. # total reward achieved in simulation
    max_expected_reward = mean[actual_best] * steps # expected reward if best action is always chosen (for assessing performance)
    steps_to_best = 0 # number of steps after which only one "best" action is always chosen
    
    # Run simulation
#    print ("Step".rjust(5), "Action".rjust(15), "Best action".rjust(15), "Reward".rjust(15), "\n")
    for step in range(steps):
    
        action = select_action(actions, step, mean_reward, times_tried) # select action to take in each step
    
        reward = random.gauss(mean[action], stdev[action]) # get reward from normal distribution for picked action
        total_reward += reward # update total reward
        mean_reward[action] = mean_reward[action] * times_tried[action] + reward # update total reward for picked action
        times_tried[action] += 1 # update number of times the action was taken
        mean_reward[action] /= times_tried[action] # update mean reward for picked action
    
        if mean_reward[best] < max(mean_reward): # if some action has better mean_reward than the assumed "best" action
            best = mean_reward.index(max(mean_reward)) # then assume that action to be the "best"
    
        if best != actual_best: # has the best action been found yet?
            steps_to_best = step + 1

#        print ("{:5d} {:15d} {:15d} {:15.8f}".format(step, action, best, reward))

    return best, actual_best, steps_to_best, total_reward, max_expected_reward, mean, stdev

In [5]:
# Run simulation once
best, actual_best, steps_to_best, total_reward, max_expected_reward,mean, stdev = run_simulation(actions, steps)

print ("Action".rjust(7), "Reward mean".rjust(15), "Reward stdev".rjust(15), "\n")
for action in range(actions):
    print ("{:7d} {:15.8f} {:15.8f}".format(action, mean[action], stdev[action]))

print ("\n Best action predicted after {:5d} steps: {:4d}".format(steps, best))
print (" Actual best action: {:4d}".format(actual_best))
print (" Steps to settle on best action: {:4d}".format(steps_to_best))
print (" Percentage of maximum expected reward achieved: {:15.8f}".format(total_reward/max_expected_reward*100))

 Action     Reward mean    Reward stdev 

      0      0.12826401      0.56968438
      1      0.71114154      0.47123216
      2      0.69288725      0.71698117
      3      0.76338243      0.38228483
      4      0.07801450      0.32566293
      5      0.33454716      0.08071681
      6      0.59606433      0.02033614
      7      0.49372509      0.87122755
      8      0.34871237      0.54730230
      9      0.26310700      0.28388342

 Best action predicted after  5000 steps:    3
 Actual best action:    3
 Steps to settle on best action:  135
 Percentage of maximum expected reward achieved:     93.35568375


In [6]:
# Function to run simulation repeatedly
def run_repeatedly(actions, steps, simulations):

    corrects = [] # List of outcomes for individual simulations: best action predicted? (True/False)
    steps_to_predict = [] # Steps it took to converge on best action (== steps if hadn't converged to best action)
    fracs_max_reward = [] # Fractions of maximum expected reward achieved
    
    correct = 0. # fraction of simulations that converged on best action
    for simulation in range(simulations):
        best, actual_best, steps_to_best, total_reward, max_expected_reward, _, _ = run_simulation(actions, steps)
        if best == actual_best: # have we predicted the actual "best" action correctly?
            correct += 1
            corrects.append(True)
        else:
            corrects.append(False)
        steps_to_predict.append(steps_to_best) # how many steps did we take before we started choosing the "best" action consistently for each simulation?
        fracs_max_reward.append(total_reward/max_expected_reward) # what fraction of the maximum expected reward did we achieve in each simulation?
    correct /= simulations # percentage of simulations that correctly chose the best action
    
    return correct, corrects, steps_to_predict, fracs_max_reward

In [7]:
# Run simulation repeatedly (may take some time)
correct, corrects, steps_to_predict, fracs_max_reward = run_repeatedly(actions, steps, simulations)

steps_successful = [] # numbers of steps to converge on correct answer (for successful runs)
print ("Correct?".rjust(10), "Steps to predict".rjust(18), "% max expected reward".rjust(23), "\n")
for simulation in range(simulations):
    print ("{:10s} {:18d} {:23.8f}".format(str(corrects[simulation]).rjust(10), steps_to_predict[simulation], fracs_max_reward[simulation]*100))
    if corrects[simulation] == True:
        steps_successful.append(steps_to_predict[simulation])

  Correct?   Steps to predict   % max expected reward 

      True               2575             92.83969392
      True                 98             94.50054145
      True               1633             94.05257787
      True                807             91.60951457
      True               3485             95.29474357
      True                435             93.50014603
      True                 17             92.61959880
      True                593             94.80058027
      True                123             91.25330938
      True                 86             94.86164008
     False               5000             92.22587800
      True               3336             95.13142904
      True                150             94.62596156
      True                664             93.25996776
      True                376             94.66210431
      True                 10             94.65079491
      True                732             92.84143104
      True                

In [8]:
# Analyze results

print ("\nPercent correct: {:15.8f}".format(correct*100))
steps_successful = np.asarray(steps_successful)
print ("Mean number of steps to converge on correct answer (if converged): {:15.8f}".format(np.mean(steps_successful)))
print ("Standard deviation for the number of steps to converge on correct answer: {:15.8f}".format(np.std(steps_successful)))
fracs_max_reward = np.asarray(fracs_max_reward)
print ("Mean percentage of maximum expected reward achieved: {:15.8f}".format(np.mean(fracs_max_reward)*100))
print ("Standard deviation for percentage of maximum expected reward achieved: {:15.8f}".format(np.std(fracs_max_reward)*100))


Percent correct:     93.50000000
Mean number of steps to converge on correct answer (if converged):    699.00427807
Standard deviation for the number of steps to converge on correct answer:   1106.57156034
Mean percentage of maximum expected reward achieved:     93.90756888
Standard deviation for percentage of maximum expected reward achieved:      1.59914708
